In [92]:
import py_trees
import behaviours as be
from py_trees.blackboard import Blackboard
import pandas as pd
from scipy import spatial
from sklearn.cluster import KMeans
import numpy as np
from pandas import DataFrame
from annotator import annotator
import re
import matplotlib.pyplot as plt

In [93]:
# read labeled cases
docu = '/Users/sileshu/Desktop/EMSdata/RAA_3658.xlsx'
df = pd.read_excel(docu)
su = 0
protocol = {}
narratives = df['Narrative']
vitals = df['Vitals']
inters = df['Interventions']
gt = []
pts = set()
for item in df['HAYDON LABEL']:
    if not pd.isnull(item):
        gt.append(item)
        pts.add(item)

intermapping= {
    '12-lead ecg':['cardiac monitor'],
    'albuterol':['albuterol'],
    'aspirin':['aspirin'],
    'assist ventilation (bvm)':['bag valve mask ventilation'],
    'capnography (1) first reading':['capnography'],
    'capnography (2) seco':['capnography'],
    'capnography (2) second reading':['capnography'],
    'capnography (3) final reading':['capnography'],
    'cardiac monitor':['cardiac monitor'],
    'nasopharyngeal airway insertio':['nasopharyngeal airway'],
    'cpap':['cpap'],
    'dexamethasone (decadron)':['dexamethasone'],
    'dextrose 10%':['dextrose'],
    'dextrose 25%':['dextrose'],
    'dextrose 50%':['dextrose'],
    'dextrose 5% in 0.45% ns':['dextrose'],
    'duoneb':['albuterol','ipratropium'],
    'fentanyl':['fentanyl'],
    'glucagon':['glucagon'],
    'glutose':['oral glucose'],
    'hospital':['transport'],
    'hospital contact':['transport'],
    'intubation':['endotracheal tube'],
    'ipratropium (atrovent)':['ipratropium'],
    'iv':['normal saline'],
    'magnesium sulfate':['morphine sulfate'],
    'midazolam (versed)':['midazolam'],
    'naloxone (narcan)':['narcan'],
    'nitroglycerine':['nitroglycerine'],
    'normal saline':['normal saline'],
    'ondansetron (zofran)':['ondansetron'],
    'oral glucose':['oral glucose'],
    'oxygen':['oxygen'],
    'restraints':['physical restraint'],
    'suction':['suction the oropharynx','suction the nasopharynx'],
}
interventions = []
for item in inters:
    inter = item.strip('{}').split('}{')
    inter = [i.split(':')[-1].strip().lower() for i in inter]
    c_int = []
    for j in inter:
        if j in intermapping:
            c_int += intermapping[j]
        else:
            print j
    interventions.append(c_int)


oropharyngeal airway insertion
bleeding controlled
pulse oximetry
oropharyngeal airway insertion
oropharyngeal airway insertion
cpr
oropharyngeal airway insertion
oropharyngeal airway insertion
oropharyngeal airway insertion
spinal motion restriction
spinal motion restriction
adenosine (adenocard)
oropharyngeal airway insertion
oropharyngeal airway insertion
travel
open airway
open airway
open airway
oropharyngeal airway insertion
open airway
open airway
open airway
open airway
oropharyngeal airway insertion
cpr
rosc
travel
open airway
pulse oximetry
oropharyngeal airway insertion
spinal motion restriction
open airway
open airway
oropharyngeal airway insertion
oropharyngeal airway insertion
oropharyngeal airway insertion
ziprasidone (geodon)
open airway
open airway
oropharyngeal airway insertion
oropharyngeal airway insertion
oropharyngeal airway insertion
open airway
pulse oximetry
oropharyngeal airway insertion
oropharyngeal airway insertion
oropharyngeal airway insertion
oropharynge

In [ ]:
from tqdm import tqdm_notebook as tqdm
EKGset = set()
pred_int = []
EKGdic = {
     '':'',
     'AV_Block_1st_Deg':'AV_Block-1st_Degree',
     'AV_Block_1st_Degree':'AV_Block-1st_Degree',
     'AV_Block_2nd_Degree_Type_1':'AV_Block_2nd_Degree_Type_1',
     'AV_Block_2nd_Degree_Type_2':'AV_Block_2nd_Degree_Type_2',
     'AV_Block_3rd_Degree':'AV_Block_3rd_Degree',
     'Asystole':'Asystole',
     'Artifact':'Artifact',
     'Atrial_Fibrill':'Atrial_Fibrillation',
     'Atrial_Fibrillation':'Atrial_Fibrillation',
     'Atrial_Flutter':'Atrial_Flutter',
     'Juncti':'Junctional_Rhythm',
     'Junctiona':'Junctional_Rhythm',
     'Junctional':'Junctional_Rhythm',
     'Other_(Not_Listed)':'Other_(Not_Listed)',
     'P': 'Paced_Rhythm',
     'PEA':'Pulseless_Electrical_Activity',
     'Pac':'Paced_Rhythm',
     'Paced_Rhythm':'Paced_Rhythm',
     'Premature_Ventricular_Contractions':'Premature_Ventricular_Contractions',
     'Right_Bundle_Branch_Block':'Right_Bundle_Branch_Block',
     'Left_Bundle_Branch_Block':'Left_Bundle_Branch_Block',
     'STEMI_Anterior_Ischemia':'STEMI_Anterior_Ischemia',
     'STEMI_Lateral_Ischemia':'STEMI_Lateral_Ischemia',
     'STEMI_Inferior_Ischemia':'STEMI_Inferior_Ischemia',
     'S':'Sinus_Rhythm',
     'Si':'Sinus_Rhythm',
     'Sin':'Sinus_Rhythm',
     'Sinu':'Sinus_Rhythm',
     'Sinus':'Sinus_Rhythm',
     'Sinus_':'Sinus_Rhythm',
     'Sinus_Arrhythmia':'Sinus_Arrhythmia',
     'Sinus_Bradycardia':'Sinus_Bradycardia',
     'Sinus_R':'Sinus_Rhythm',
     'Sinus_Rh':'Sinus_Rhythm',
     'Sinus_Rhy':'Sinus_Rhythm',
     'Sinus_Rhyth':'Sinus_Rhythm',
     'Sinus_Rhythm':'Sinus_Rhythm',
     'Sinus_Rhythm,Sinus_Tachycardia':'Sinus_Rhythm,Sinus_Tachycardia',
     'Sinus_T':'Sinus_Tachycardia',
     'Sinus_Tach':'Sinus_Tachycardia',
     'Sinus_Tachyc':'Sinus_Tachycardia',
     'Sinus_Tachycardi':'Sinus_Tachycardia',
     'Sinus_Tachycardia':'Sinus_Tachycardia',
     'Supravent':'Supraventricular_Tachycardia',
     'Supraventricular_Tachycardia':'Supraventricular_Tachycardia',
     'Ventricular_Fibrillation':'Ventricular_Fibrillation',
     'Ventricular_Tachycardia_(With_Pulse)':'Ventricular_Tachycardia',
     'Non_STEMI_Lateral_Ischemia':'Non_STEMI_Lateral_Ischemia'
}
# extract concept and calculate similarity
from ranking_func import rank
pool = set(['Medical - Abdominal Pain',
            'Medical - Altered Mental Status',
            'Medical - Seizure',
            'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
            'General - Behavioral/Patient Restraint',
            'Medical - Overdose/Poisoning - Opioid',
            'Medical - Diabetic - Hypoglycemia',
            'Medical - Chest Pain - Cardiac Suspected'])

def pre_tick_handler(behaviour_tree):
    blackboard = Blackboard()
    blackboard.tick_num += 1
results = []
scores = []
ConceptRes = []
for i,item in enumerate(tqdm(narratives)):
    if not pd.isnull(vitals[i]):
        vt = vitals[i].strip('{}').split('}{')
        vt = [it.split(':')[-1] for it in vt]
        for idx,it in enumerate(vt):
            if 'EKG-' in it:
                temp = it.split('EKG-')
                temp[1] = temp[1].replace(' ','_')
                temp[1] = temp[1].replace('-','_')
                if ',' in temp[1]:
                    t = temp[1].split(',')
                    t = [EKGdic[i] for i in t]
                    vt[idx] = temp[0] + 'EKG-' + ','.join(t)
                else:
                    vt[idx] = temp[0] + 'EKG-' + EKGdic[temp[1]]
                if len(temp[1]) > 0:
                    EKGset.add(temp[1])
        vt = [ite for l in vt for ite in l.strip().split(' ')]
        for idx in xrange(len(vt)):
            if idx < len(vt) and '-' not in vt[idx]:
                vt.pop(idx)
        for idx,it in enumerate(vt):
            temp = it.split('-')
            vt[idx] = (temp[0],temp[1])
    blackboard = Blackboard()
    blackboard.text = [item]
    root = py_trees.composites.Sequence("Root_1")
    IG = be.InformationGathering(inC = vt)
    TC = be.TextCollection()
    V = be.Vectorize()
    PS = be.ProtocolSelector()
    root.add_children([TC,IG,V,PS,be.protocols])
    behaviour_tree = py_trees.trees.BehaviourTree(root)
    behaviour_tree.add_pre_tick_handler(pre_tick_handler)
    behaviour_tree.setup(15)
    behaviour_tree.tick_tock(
            sleep_ms=50,
            number_of_iterations=1,
            pre_tick_handler=None,
       post_tick_handler=None
        )
    scores.append(blackboard.ranking)
    ConceptRes.append(blackboard.ConcLog)
    results.append(blackboard.candi)
    res = []
    for key in blackboard.feedback:
        if blackboard.feedback[key] > 0.1:
            res.append((key, blackboard.feedback[key]))
    pred_int.append(res)
    

(transport;True;transported;transported;726.0;1)
(narcan;True;narcan;narcan;861.0;1)
(transport;True;transported;transported;714.0;1)
(narcan;True;narcan;narcan;578.0;1)
(capnography;True;capnography;capnography;1000.0;1)
(transport;True;transferred;transferred;790.0;1)
(normal saline;True;normal saline;normal saline;861.0;1)
(narcan;True;narcan;narcan;629.0;1)
(capnography;True;capnography;capnography;861.0;1)
(morphine;True;heroin;heroin;593.0;1)
(cardiac monitor;True;12;cardiac monitor;1000.0;1)
(transport;True;transferred;transferred;770.0;1)
(narcan;True;narcan;narcan;753.0;1)
(capnography;True;capnography;capnography;660.0;1)
(morphine;True;heroin;heroin;1000.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;833.0;1)
(oxygen;True;15;oxygen;790.0;1)
(transport;True;transported;transported;737.0;1)
(narcan;True;narcan;narcan;1000.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;861.0;1)
(transport;True;transferred;transferred;748.0;1)
(supine position;True;supine

(transport;True;transport;transport;770.0;1)
(supine position;True;lying;lying;739.0;1)
(narcan;True;narcan;narcan;581.0;1)
(capnography;True;18;capnography;861.0;1)
(morphine;True;heroin;heroin;1000.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;1000.0;1)
(oxygen;True;12;o2;623.0;1)
(transport;True;transported;transported;966.0;1)
(transport;True;transferred;transferred;742.0;1)
(supine position;True;supine position;supine position;790.0;1)
(nasopharyngeal airway;True;nasopharyngeal airway;nasopharyngeal airway;770.0;1)
(capnography;True;capnography monitoring;capnography monitoring;901.0;1)
(morphine;True;heroin;heroin;770.0;1)
(cardiac monitor;True;4;cardiac monitor;1000.0;1)
(oxygen;True;20;oxygen;790.0;1)
(transport;True;transferred;transferred;770.0;1)
(supine position;True;lying;lying;770.0;1)
(narcan;True;narcan;narcan;581.0;1)
(capnography;True;capnography;capnography;660.0;1)
(morphine;True;heroin;heroin;1000.0;1)
(cardiac monitor;True;cardiac monitor;cardiac moni

(transport;True;2;transported;756.0;1)
(supine position;True;supine position;supine position;806.0;1)
(narcan;True;0.5;narcan;604.0;1)
(morphine;True;heroin;heroin;694.0;1)
(cardiac monitor;True;12;cardiac monitor;1000.0;1)
(oxygen;True;o2;o2;861.0;1)
(transport;True;transported;transported;719.0;1)
(supine position;True;supine;supine;790.0;1)
(narcan;True;narcan;narcan;604.0;1)
(morphine;True;heroin;heroin;586.0;1)
(transport;True;transferred;transferred;790.0;1)
(normal saline;True;normal saline;normal saline;623.0;1)
(supine position;True;supine position;supine position;833.0;1)
(narcan;True;narcan;narcan;790.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;640.0;1)
(oxygen;True;5;o2;812.0;1)
(transport;True;transferred;transferred;770.0;1)
(narcan;True;narcan;narcan;861.0;1)
(morphine;True;heroin;heroin;604.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;1000.0;1)
(oxygen;True;oxygen;oxygen;1000.0;1)
(narcan;True;narcan;narcan;660.0;1)
(capnography;True;2;capno

(normal saline;True;normal saline;normal saline;666.0;1)
(capnography;True;capnography;capnography;861.0;1)
(morphine;True;heroin;heroin;660.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;1000.0;1)
(transport;True;transported;transported;737.0;1)
(supine position;True;supine;supine;748.0;1)
(narcan;True;1;narcan;626.0;1)
(morphine;True;heroin;heroin;593.0;1)
(transport;True;transferred;transferred;770.0;1)
(morphine;True;heroin;heroin;593.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;1000.0;1)
(oxygen;True;oxygen;oxygen;1000.0;1)
(transport;True;transferred;transferred;790.0;1)
(midazolam;True;midazolam;midazolam;827.0;1)
(supine position;True;supine;supine;790.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;1000.0;1)
(oxygen;True;o2;o2;861.0;1)
(transport;True;transport;transport;1000.0;1)
(narcan;True;1525;narcan;581.0;1)
(nasopharyngeal airway;True;nasal airway;nasal airway;1000.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;806.0;1)
(

(transport;True;transported;transported;756.0;1)
(supine position;True;supine;supine;790.0;1)
(narcan;True;narcan;narcan;570.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;790.0;1)
(transport;True;transferred;transferred;770.0;1)
(narcan;True;narcan;narcan;1000.0;1)
(morphine;True;heroin;heroin;790.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;1000.0;1)
(oxygen;True;oxygen;oxygen;1000.0;1)
(transport;True;transported;transported;966.0;1)
(morphine;True;heroin;heroin;1000.0;1)
(oxygen;True;o2;o2;770.0;1)
(transport;True;transported;transported;966.0;1)
(supine position;True;lying;lying;1000.0;1)
(transport;True;transferred;transferred;770.0;1)
(narcan;True;narcan;narcan;623.0;1)
(morphine;True;heroin;heroin;770.0;1)
(transport;True;carried;carried;790.0;1)
(supine position;True;lying;lying;770.0;1)
(narcan;True;2;narcan;744.0;1)
(physical restraint;True;restraints;restraints;770.0;1)
(capnography;True;capnography;capnography;660.0;1)
(cardiac monitor;True;cardia

...

(transport;True;transport;transport;604.0;1)
(supine position;True;supine;supine;770.0;1)
(dextrose;True;dextrose;dextrose;581.0;1)
(transport;True;transport;transport;1000.0;1)
(supine position;True;lying;lying;861.0;1)
(transport;True;transported;transported;737.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;861.0;1)
(dextrose;True;glucose;glucose;637.0;1)
(transport;True;transport;transport;1000.0;1)
(oral glucose;True;oral glucose;oral glucose;631.0;1)
(supine position;True;lying;lying;744.0;1)
(transport;True;transport;transport;827.0;1)
(oral glucose;True;oral glucose;oral glucose;631.0;1)
(transport;True;transport;transport;770.0;1)
(transport;True;transferred;transferred;770.0;1)
(supine position;True;lying;lying;770.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;833.0;1)
(dextrose;True;glucose;glucose;1000.0;1)
(transport;True;transport;transport;604.0;1)
(oral glucose;True;0;oral glucose;631.0;1)
(dextrose;True;glucose;glucose;861.0;1)
(transport;True;

(transport;True;transported;transported;966.0;1)
(transport;True;transport;transport;1000.0;1)
(supine position;True;lying;lying;622.0;1)
(supine position;True;supine;supine;790.0;1)
(transport;True;transport;transport;1000.0;1)
(oral glucose;True;22.;oral glucose;631.0;1)
(dextrose;True;glucose;glucose;827.0;1)
(transport;True;transport;transport;604.0;1)
(oral glucose;True;oral glucose;oral glucose;631.0;1)
(dextrose;True;2;glucose;660.0;1)
(transport;True;transferred;transferred;770.0;1)
(oral glucose;True;39.;oral glucose;631.0;1)
(supine position;True;lying;lying;739.0;1)
(transport;True;transport;transport;604.0;1)
(oral glucose;True;oral glucose;oral glucose;607.0;1)
(dextrose;True;glucose;glucose;581.0;1)
(glucagon;True;glucagon;glucagon;586.0;1)
(transport;True;transport;transport;694.0;1)
(oral glucose;True;oral glucose;oral glucose;652.0;1)
(dextrose;True;34.;glucose;660.0;1)
(transport;True;transport;transport;770.0;1)
(dextrose;True;5;glucose;637.0;1)
(glucagon;True;glucag

(transport;True;transport;transport;1000.0;1)
(oral glucose;True;oral glucose;oral glucose;631.0;1)
(cardiac monitor;True;12;cardiac monitor;833.0;1)
(transport;True;transferred;transferred;790.0;1)
(normal saline;True;normal saline;normal saline;1000.0;1)
(supine position;True;supine;supine;790.0;1)
(dextrose;True;0;dextrose;637.0;1)
(transport;True;transport;transport;753.0;1)
(dextrose;True;glucose;glucose;861.0;1)
(transport;True;transferred;transferred;770.0;1)
(cardiac monitor;True;cardiac monitoring;cardiac monitoring;1000.0;1)
(dextrose;True;glucose;glucose;861.0;1)
(glucagon;True;glucagon;glucagon;586.0;1)
(transport;True;transported;transported;966.0;1)
(oral glucose;True;oral glucose;oral glucose;631.0;1)
(supine position;True;body supine position;body supine position;648.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;833.0;1)
(oxygen;True;15;oxygen;790.0;1)
(transport;True;transferred;transferred;770.0;1)
(cardiac monitor;True;8;cardiac monitor;1000.0;1)
(transp

(transport;True;transport;transport;1000.0;1)
(dextrose;True;0;dextrose;827.0;1)
(transport;True;5;transported;719.0;1)
(oral glucose;True;oral glucose;oral glucose;631.0;1)
(transport;True;transported;transported;756.0;1)
(transport;True;transport;transport;604.0;1)
(dextrose;True;glucose;glucose;827.0;1)
(transport;True;transferred;transferred;790.0;1)
(dextrose;True;0;dextrose;660.0;1)
(oral glucose;True;glucose po;glucose po;827.0;1)
(transport;True;transport;transport;1000.0;1)
(oral glucose;True;oral glucose;oral glucose;631.0;1)
(glucagon;True;glucagon;glucagon;604.0;1)
(transport;True;transferred;transferred;1000.0;1)
(oral glucose;True;oral glucose;oral glucose;652.0;1)
(dextrose;True;39.;glucose;748.0;1)
(transport;True;transferred;transferred;790.0;1)
(dextrose;True;glucose;glucose;861.0;1)
(transport;True;transported;transported;737.0;1)
(capnography;True;capnography;capnography;1000.0;1)
(cardiac monitor;True;cardiac monitor;cardiac monitor;833.0;1)
(oxygen;True;2;oxygen;7

(transport;True;transport;transport;604.0;1)
(supine position;True;supine;supine;790.0;1)
(transport;True;transferred;transferred;1000.0;1)
(supine position;True;supine;supine;790.0;1)
(oxygen;True;oxygen;oxygen;1000.0;1)
(dextrose;True;dextrose;dextrose;581.0;1)
(transport;True;transport;transport;604.0;1)
(oxygen;True;o2;o2;660.0;1)
(transport;True;transport;transport;1000.0;1)
(oral glucose;True;oral glucose;oral glucose;754.0;1)
(dextrose;True;glucose;glucose;738.0;1)
(transport;True;transport;transport;604.0;1)
(supine position;True;lying;lying;861.0;1)
(dextrose;True;dextrose;dextrose;586.0;1)
(transport;True;transferred;transferred;770.0;1)
(dextrose;True;0;dextrose;632.0;1)
(transport;True;transferred;transferred;760.0;1)
(transport;True;transport;transport;738.0;1)
(transport;True;transported;transported;756.0;1)
(dextrose;True;dextrose;dextrose;593.0;1)
(transport;True;transferred;transferred;790.0;1)
(supine position;True;supine;supine;790.0;1)
(cardiac monitor;True;cardiac 

In [100]:
pred_int_copy = pred_int[:]
pred_int = [[i[0] for i in case] for case in pred_int_copy]
pred_pos = [[i[1] for i in case] for case in pred_int_copy]

In [102]:
tTP = 0.
tFP = 0.
tFN = 0.
fpst = dict()
fnst = dict()
fps = []
fns = []
tlength = 0

precisions = []
recalls = []
f1scores = []
wprecisions = []
wrecalls = []
wf1scores = []
for i,item in enumerate(pred_int):
    TP = 0.
    FP = 0.
    FN = 0.
    fp = []
    fn = []
    l = len(interventions[i])
    tlength += l
    for m in item:
        if m in interventions[i]:
            TP += 1
            tTP += 1
        else:
            if m in fpst:
                fpst[m] += 1
            else:
                fpst[m] = 1
            FP += 1
            tFP += 1
            fp.append(m)
    for n in interventions[i]:
        if n not in pred_int[i]:
            if n in fnst:
                fnst[n] += 1
            else:
                fnst[n] = 1
            FN += 1
            tFN += 1
            fn.append(n)
    fps.append(fp)
    fns.append(fn)
    if TP == 0 and FP == 0:
        recalls.append(TP / (TP + FN))
        precisions.append(0.)
        f1scores.append(2 * TP / (2 * TP + FN + FP))
        wrecalls.append((TP / (TP + FN)) * l)
        wprecisions.append(0.)
        wf1scores.append((2 * TP / (2 * TP + FN + FP)) * l)
        continue
    wrecalls.append((TP / (TP + FN)) * l)
    wprecisions.append((TP / (TP + FP)) * l)
    wf1scores.append((2 * TP / (2 * TP + FN + FP)) * l)

In [103]:
interset = set()
for i in interventions:
    for j in i:
        interset.add(j)
int2idx = {}
for idx,item in enumerate(interset):
    int2idx[item] = idx
inter_vec = [[0] * len(interset) for _ in range(len(interventions))]
pre_int_vec = [[0] * len(interset) for _ in range(len(pred_int))]
for idx,case in enumerate(interventions):
    for inter in case:
        inter_vec[idx][int2idx[inter]] = 1
for idx,case in enumerate(pred_int):
    for inter in case:
        if inter in int2idx:
            pre_int_vec[idx][int2idx[inter]] = 1


In [104]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print recall_score(np.array(inter_vec),np.array(pre_int_vec), average = 'weighted')
print recall_score(np.array(inter_vec),np.array(pre_int_vec), average = 'micro')
print precision_score(np.array(inter_vec),np.array(pre_int_vec), average = 'weighted')
print precision_score(np.array(inter_vec),np.array(pre_int_vec), average = 'micro')
print f1_score(np.array(inter_vec),np.array(pre_int_vec), average = 'weighted')
print f1_score(np.array(inter_vec),np.array(pre_int_vec), average = 'micro')

0.654450880719
0.654450880719
0.642116412035
0.754686115466
0.635780034069
0.701003513881


In [52]:
fnst

{'albuterol': 1226,
 'aspirin': 212,
 'bag valve mask ventilation': 269,
 'capnography': 223,
 'cardiac monitor': 54,
 'cpap': 96,
 'dexamethasone': 326,
 'dextrose': 333,
 'endotracheal tube': 4,
 'fentanyl': 78,
 'glucagon': 32,
 'ipratropium': 1100,
 'midazolam': 385,
 'morphine sulfate': 59,
 'narcan': 141,
 'nasopharyngeal airway': 163,
 'nitroglycerine': 103,
 'normal saline': 601,
 'ondansetron': 91,
 'oral glucose': 141,
 'oxygen': 650,
 'physical restraint': 79,
 'suction the nasopharynx': 36,
 'suction the oropharynx': 36,
 'transport': 5}

In [111]:
fpst

{'aspirin': 163,
 'bag valve mask ventilation': 502,
 'cardiac monitor': 387,
 'cpap': 198,
 'dexamethasone': 197,
 'dextrose': 5,
 'endotracheal tube': 1,
 'fentanyl': 19,
 'midazolam': 19,
 'narcan': 360,
 'nitroglycerin': 5,
 'normal saline': 621,
 'ondansetron': 142,
 'oral glucose': 190,
 'oxygen': 395,
 'physical restraint': 44,
 'transport': 356}

In [107]:
df['FN'] = fns
df['FP'] = fps
df['predictions'] = pred_int_copy
df['pre_protocols'] = results
df['similarities'] = scores

In [108]:
df.to_excel('RAA_3657.xlsx',index = False)

In [109]:
bina = []
for i,r in enumerate(results):
    bina.append(gt[i] in r[:3])
print float(sum(bina))/len(bina)
        

0.889526934646


In [110]:
sum(bina)

3253

In [23]:
Inters = []
for item in interventions1:
    inter = item.strip('{}').split('}{')
    inter = [i.split(':')[-1].strip().lower() for i in inter]
    Inters.append(inter)
interset = set()
for i in Inters:
    for j in i:
        interset.add(j)

In [112]:
FN2H = {
 '': 0,
 'aspirin': 1,
 'albuterol': 1,
 'bag valve mask ventilation': 3,
 'capnography': 1,
 'cardiac monitor': 1,
 'cpap': 3,
 'dexamethasone': 1,
 'dextrose': 3,
 'fentanyl': 0,
 'glucagon': 2,
 'ipratropium': 1,
 'midazolam': 3,
 'morphine sulfate': 0,
 'narcan': 3,
 'nasopharyngeal airway': 3,
 'nitroglycerine': 2,
 'normal saline': 0,
 'ondansetron': 0,
 'oral glucose': 1,
 'oxygen': 2,
 'physical restraint': 2,
 'transport': 0,
 'suction the oropharynx':2,
 'suction the nasopharynx':2
}

FP2H = {
 '': 0,
 'aspirin': 1,
 'bag valve mask ventilation': 1,
 'cardiac monitor': 0,
 'cpap': 2,
 'dexamethasone': 1,
 'dextrose': 1,
 'midazolam': 3,
 'narcan': 1,
 'nitroglycerin': 1,
 'normal saline': 1,
 'ondansetron': 0,
 'oral glucose': 1,
 'oxygen': 0,
 'physical restraint': 1,
 'transport': 0
}

In [113]:
res = pd.read_excel('/Users/sileshu/Desktop/BT/RAA_3657.xlsx')

In [131]:
hs = []
for i in res['FN']:
    sc = [FN2H[it.strip().strip('\'')] for it in i.strip('[]').split(',')]
    hs.append(max(sc))
for idx,i in enumerate(res['FP']):
    sc = [FP2H[it.strip().strip('\'')] for it in i.strip('[]').split(',')]
    if max(sc) > hs[idx]:
        hs[idx] = max(sc)


In [132]:
print 'red:' + str(hs.count(3))
print 'orange:' + str(hs.count(2))
print 'yellow:' + str(hs.count(1))
print 'green:' + str(hs.count(0))

red:25
orange:25
yellow:73
green:43


In [5]:
from tqdm import tqdm_notebook as tqdm
EKGset = set()
pred_int = []
EKGdic = {
     '':'',
     'AV_Block_1st_Deg':'AV_Block-1st_Degree',
     'AV_Block_1st_Degree':'AV_Block-1st_Degree',
     'AV_Block_1st_Degree,Sinus_Rhythm':'AV_Block-1st_Degree,Sinus_Rhythm',
     'AV_Block_2nd_Degree_Type_1':'AV_Block_2nd_Degree_Type_1',
     'Asystole':'Asystole',
     'Artifact':'Artifact',
     'Atrial_Fibrill':'Atrial_Fibrillation',
     'Atrial_Fibrillation':'Atrial_Fibrillation',
     'Atrial_Flutter':'Atrial_Flutter',
     'Juncti':'Junctional_Rhythm',
     'Junctiona':'Junctional_Rhythm',
     'Junctional':'Junctional_Rhythm',
     'Other_(Not_Listed)':'Other_(Not_Listed)',
     'P': 'Paced_Rhythm',
     'PEA':'Pulseless_Electrical_Activity',
     'Pac':'Paced_Rhythm',
     'Paced_Rhythm':'Paced_Rhythm',
     'Premature_Ventricular_Contractions,,Sinus_Rhythm':'Premature_Ventricular_Contractions,Sinus_Rhythm',
     'Premature_Ventricular_Contractions,Sinus_Rhythm':'Premature_Ventricular_Contractions,Sinus_Rhythm',
     'Right_Bundle_Branch_Block':'Right_Bundle_Branch_Block',
     'STEMI_Anterior_Ischemia':'STEMI_Anterior_Ischemia',
     'STEMI_Lateral_Ischemia':'STEMI_Lateral_Ischemia',
     'S':'Sinus_Rhythm',
     'Si':'Sinus_Rhythm',
     'Sin':'Sinus_Rhythm',
     'Sinu':'Sinus_Rhythm',
     'Sinus':'Sinus_Rhythm',
     'Sinus_':'Sinus_Rhythm',
     'Sinus_Arrhythmia':'Sinus_Arrhythmia',
     'Sinus_Arrhythmia,Sinus_Rhythm':'Sinus_Arrhythmia,Sinus_Rhythm',
     'Sinus_Arrhythmia,Sinus_Tachycardia':'Sinus_Arrhythmia,Sinus_Tachycardia',
     'Sinus_Bradycardia':'Sinus_Bradycardia',
     'Sinus_R':'Sinus_Rhythm',
     'Sinus_Rh':'Sinus_Rhythm',
     'Sinus_Rhy':'Sinus_Rhythm',
     'Sinus_Rhyth':'Sinus_Rhythm',
     'Sinus_Rhythm':'Sinus_Rhythm',
     'Sinus_T':'Sinus_Tachycardia',
     'Sinus_Tach':'Sinus_Tachycardia',
     'Sinus_Tachyc':'Sinus_Tachycardia',
     'Sinus_Tachycardi':'Sinus_Tachycardia',
     'Sinus_Tachycardia':'Sinus_Tachycardia',
     'Supravent':'Supraventricular_Tachycardia',
     'Supraventricular_Tachycardia':'Supraventricular_Tachycardia',
     'Ventricular_Fibrillation':'Ventricular_Fibrillation',
     'Ventricular_Tachycardia_(With_Pulse)':'Ventricular_Tachycardia'
}
# extract concept and calculate similarity
from ranking_func import rank
pool = set(['Medical - Abdominal Pain',
            'Medical - Altered Mental Status',
            'Medical - Seizure',
            'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
            'General - Behavioral/Patient Restraint',
            'Medical - Overdose/Poisoning - Opioid',
            'Medical - Diabetic - Hypoglycemia',
            'Medical - Chest Pain - Cardiac Suspected'])

def pre_tick_handler(behaviour_tree):
    blackboard = Blackboard()
    blackboard.tick_num += 1
results = []
scores = []
ConceptRes = []
for i,item in enumerate(tqdm(narratives)):
    vt = vitals[i].strip('{}').split('}{')
    vt = [it.split(':')[-1] for it in vt]
    for idx,it in enumerate(vt):
        if 'EKG-' in it:
            temp = it.split('EKG-')
            temp[1] = temp[1].replace(' ','_')
            temp[1] = temp[1].replace('-','_')
            vt[idx] = temp[0] + 'EKG-' + EKGdic[temp[1]]
            if len(temp[1]) > 0:
                EKGset.add(temp[1])
    vt = [ite for l in vt for ite in l.strip().split(' ')]
    for idx in xrange(len(vt)):
        if idx < len(vt) and '-' not in vt[idx]:
            vt.pop(idx)
    for idx,it in enumerate(vt):
        temp = it.split('-')
        vt[idx] = (temp[0],temp[1])
    blackboard = Blackboard()
    blackboard.text = item
    blackboard.case_num = i+1
    root = py_trees.composites.Sequence("Root_1")
    IG = be.IG(slist = '/Users/sileshu/Downloads/Archive/WithUnigram/50candidateConcepts_EMS.txt', \
                                 inC = vt, neg_res = '/Users/sileshu/Downloads/all_results.txt')
    TC = be.TextCollection()
    V = be.Vectorize()
    PS = be.ProtocolSelector()
    root.add_children([TC,IG,V,PS,be.protocols])
    behaviour_tree = py_trees.trees.BehaviourTree(root)
    behaviour_tree.add_pre_tick_handler(pre_tick_handler)
    behaviour_tree.setup(15)
    behaviour_tree.tick_tock(
            sleep_ms=50,
            number_of_iterations=1,
            pre_tick_handler=None,
       post_tick_handler=None
        )
    scores.append(blackboard.ranking)
   # ConceptRes.append(blackboard.ConcLog)
    results.append(blackboard.candi)
    res = []
    for key in blackboard.feedback:
        if blackboard.feedback[key] > 0.1:
            res.append(key)
    pred_int.append(res)    

In [6]:
TP = 0.
FP = 0.
FN = 0.
fpst = dict()
fnst = dict()
fps = []
fns = []

precisions = []
recalls = []
f1scores = []
for i,item in enumerate(pred_int):
    TP = 0.
    FP = 0.
    FN = 0.
    fp = []
    fn = []
    for m in item:
        if m in interventions[i]:
            TP += 1
        else:
            if m in fpst:
                fpst[m] += 1
            else:
                fpst[m] = 1
            FP += 1
            fp.append(m)
    for n in interventions[i]:
        if n not in pred_int[i]:
            if n in fnst:
                fnst[n] += 1
            else:
                fnst[n] = 1
            FN += 1
            fn.append(n)
    fps.append(fp)
    fns.append(fn)
    if TP == 0 and FP == 0:
        recalls.append(TP / (TP + FN))
        precisions.append(0.)
        f1scores.append(2 * TP / (2 * TP + FN + FP))
        continue
    recalls.append(TP / (TP + FN))
    precisions.append(TP / (TP + FP))
    f1scores.append(2 * TP / (2 * TP + FN + FP))

In [7]:
ct = 0
fnc = 0
fpc = 0
for i in range(len(fns)):
    if len(fns[i]) == 0 and len(fps[i]) == 0:
        ct += 1
    if len(fns[i]) == 0:
        fpc += 1
    if len(fps[i]) == 0:
        fnc += 1
        
ct,fnc,fpc

(25, 65, 83)

In [8]:
print 'M-precision:' + '%.3f'%(sum(precisions)/len(precisions))
print 'M-recall:' + '%.3f'%(sum(recalls)/len(recalls))
print 'M-F1-score:' + '%.3f'%(sum(f1scores)/len(f1scores))

M-precision:0.756
M-recall:0.778
M-F1-score:0.737


In [9]:
bina = []
for i,r in enumerate(results):
    bina.append(gt[i] in r[:3])
print float(sum(bina))/len(bina)

0.656626506024


In [10]:
df['FN'] = fns
df['FP'] = fps

In [11]:
df.to_excel('RAA_166_25candidateConcepts_EMS_wU.xlsx',index = False)

In [21]:
res = pd.read_excel('/Users/sileshu/Desktop/BT/RAA_286_Reduced_classes.xlsx')

In [22]:
FN2H = {
 '': 0,
 'aspirin': 1,
 'albuterol': 1,
 'bag valve mask ventilation': 3,
 'capnography': 1,
 'cardiac monitor': 1,
 'cpap': 3,
 'dexamethasone': 1,
 'dextrose': 3,
 'fentanyl': 0,
 'glucagon': 2,
 'ipratropium': 1,
 'midazolam': 3,
 'morphine sulfate': 0,
 'narcan': 3,
 'nasopharyngeal airway': 3,
 'nitroglycerine': 2,
 'normal saline': 0,
 'ondansetron': 0,
 'oral glucose': 1,
 'oxygen': 2,
 'physical restraint': 2,
 'transport': 0,
 'suction the oropharynx':2,
 'suction the nasopharynx':2
}

FP2H = {
 '': 0,
 'aspirin': 1,
 'bag valve mask ventilation': 1,
 'cardiac monitor': 0,
 'cpap': 2,
 'dexamethasone': 1,
 'dextrose': 1,
 'midazolam': 3,
 'narcan': 1,
 'nitroglycerin': 1,
 'normal saline': 1,
 'ondansetron': 0,
 'oral glucose': 1,
 'oxygen': 0,
 'physical restraint': 1,
 'transport': 0
}

hs = []
for i in res['FN']:
    sc = [FN2H[it.strip().strip('\'')] for it in i.strip('[]').split(',')]
    hs.append(max(sc))
for idx,i in enumerate(res['FP']):
    sc = [FP2H[it.strip().strip('\'')] for it in i.strip('[]').split(',')]
    if max(sc) > hs[idx]:
        hs[idx] = max(sc)
        
print 'red:' + str(hs.count(3))
print 'orange:' + str(hs.count(2))
print 'yellow:' + str(hs.count(1))
print 'green:' + str(hs.count(0))

KeyError: u'cardiac monitor & trans'